-----
## PID
Basic PID unit.
The PID output follows the formulas:
$$e(t) = x(t) - y(t) \\
u(t) = K_p e(t) + K_i \sum_{\tau=0}^t e(\tau)\Delta t + K_d (e(t) - e(t-\Delta t))$$
where $x(t)$ is the setpoint, $y(t)$ is the feedback value, $e(t)$ is the error, $u(t)$ is the PID output, $\Delta t$ is the simulation time step width and $K_p$, $K_i$ and $K_d$ are the PID gain of the proportional, integral and differential respectively.

In [1]:
class PID:
    """Basic PID unit for the simulation."""
    
    def __init__(self, dt, Kp=0.2, Ki=0.0, Kd=0.0):
        """Generates PID controller unit.
        
        Parameters
        ----------
        dt : float
            The simulation time step width [sec]
            
        Kp : float
            Proportional gain
            
        Ki : float
            Integral gain
            
        Kd : float
            Differential gain
            
        """
        
        self.dt = dt
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        
        # Integral of error
        self.Ei = 0.0
        # Last error
        self.Ep = 0.0
    
    def update(self, setpoint, feedback):
        """Returns PID output.
        
        Parameters
        ----------
        setpoint : float
            Setpoint
            
        feedback : float
            Feedback value
        """
        
        error = setpoint - feedback
        
        # integralation
        self.Ei += error * self.dt
        # differentiation
        Ed = error - self.Ep
        
        # output
        u = self.Kp * error + self.Ki * self.Ei + self.Kd * Ed
        
        # update past error
        self.Ep = error
        
        return u

### PID unit test
PID control of the vertical velocity of the Hexacopter.

In [1]:
import matplotlib.pyplot as plt
import HxCopter.hexacopter
#import Hexacopter

# Display plots inline and change default figure size
%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 4.0)

# simulation time step width
dt = 0.01
# simulation time [sec]
runtime = 15
# number of the simulation time step
num_step = int(runtime / dt)
print(num_step)

# initial rotor speed [rpm]
omega0 = 386.4
# generate the hexacopter model
hx = Hexacopter(dt, omega0)
#generate the PID controller
#pid = PID(dt, Kp=5.0, Ki=3.5, Kd=1.2)
pid = PID(dt, Kp=0.15, Ki=0.15, Kd=0.1)
# mass [kg]
m = 2.1
# propeller thrust [rad^2/sec^2]
kT = 2.3 * 10 ** (-5)

ew = - m / 6 / kT
print('Ew = ',ew)

# log
data_w = np.zeros(num_step)
data_o = np.zeros(num_step)
cmds = np.zeros(num_step)


# ========
# COMMAND
# ========
cmd = 0.0

for t in range(num_step):
    w = hx.get_velocity()[0]
    # register the log
    data_w[t] = w
    # PID calculation
    if t == int(3 / dt):
        cmd = 0.2
    if t == int(6 /dt):
        cmd = 0.
    if t == int(9 / dt):
        cmd = 0.2
    if t == int(12 /dt):
        cmd = 0.
    cmds[t] = cmd
    u = pid.update(cmd, w) * ew
    data_o[t] = w
    us = np.array([u, u, u, u, u, u])
    
    # command imput
    hx.set_rotor_speed(us)
    # model update
    hx.update()

# draw graph of vertical speed log
t = np.arange(0, runtime, dt)
#cmds = np.ones(num_step) * cmd
plt.plot(t, cmds)
plt.plot(t, data_w)
plt.ylim(-0.2, 0.6)
plt.xlim(0, 15)

ModuleNotFoundError: No module named 'HxCopter'